In [ ]:
RESULTS_FOLDER = "/path/to/results/folder"

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats
matplotlib.rc('font', family='serif', size=24)
matplotlib.rc('errorbar', capsize=3)

In [ ]:
suffix = os.path.join("inv_lambda-1.0_epsilon-0.01_apply_dp-True", "data_dp_round_1.npz")
def _load_data(exp_name, attack_type):
    return {
        1: np.load(os.path.join(RESULTS_FOLDER, exp_name, f"split_attack_{attack_type}_50_one_validation_fraction_0.2", suffix)),
        12: np.load(os.path.join(RESULTS_FOLDER, exp_name, f"split_attack_{attack_type}_50_quarter_validation_fraction_0.2",  suffix)),
        24: np.load(os.path.join(RESULTS_FOLDER, exp_name, f"split_attack_{attack_type}_50_half_validation_fraction_0.2", suffix)),
    }
adult_random = _load_data("adult", "random")
kdd99_random = _load_data("kdd99", "random")
adult_inverted = _load_data("adult", "inverted")
kdd99_random = _load_data("kdd99", "inverted")

In [ ]:
def plot(ax, label, client_data, ylabel: bool = False, legend = False):
    ax.set_title(label)
    ax.grid(which='both', axis='y')
    ax.grid(which='major', axis='x')
    ax.minorticks_on()
    
    ax.set_xlabel("# of\n Malicious Agents")
    ax.set_xbound(0, 25)
    ax.set_xticks([1, 12, 24], minor=False)
    ax.xaxis.set_minor_formatter(plt.NullFormatter())
    ax.set_autoscalex_on(False)
    # ax.set_xticklabels(['{:.2f}'.format(x) for x in ax.get_xticks()])
    
    if ylabel:
        ax.set_ylabel("Overall\n Score ($p_k$)")
    ax.set_ybound(0.0, 1.0)
    ax.set_yticks([0, 0.5, 1.0], minor=False)
    ax.set_yticks([0.25, 0.75], minor=True)
    ax.set_autoscaley_on(False)
    ax.set_yticklabels(['{:.1f}'.format(x) for x in ax.get_yticks()])

    fmt = ['gD', 'ms']
    
    xs = list(key for key in client_data.keys())
    special_means_pct = []
    special_lower_pct_errors = []
    special_upper_pct_errors = []
    normal_means_pct = []
    normal_lower_pct_errors = []
    normal_upper_pct_errors = []
    for num_special_clients, data in client_data.items():
        all_scores = np.squeeze(data["overall_score"])
        normal_y = all_scores[:-num_special_clients]
        special_y = all_scores[-num_special_clients:]
        print(scipy.stats.ttest_ind(special_y, normal_y))
        for y, means_pct, lower_pct_errors, upper_pct_errors in ((special_y, special_means_pct, special_lower_pct_errors, special_upper_pct_errors),
                  (normal_y, normal_means_pct, normal_lower_pct_errors, normal_upper_pct_errors)):
            mean = np.mean(y)
            mean_pct = scipy.stats.percentileofscore(all_scores, mean) / 100
            means_pct.append(mean)
            std = np.std(y)
            lower_pct = scipy.stats.percentileofscore(all_scores, mean - std) / 100
            lower_pct_errors.append(std)
            upper_pct = scipy.stats.percentileofscore(all_scores, mean + std) / 100
            upper_pct_errors.append(std)
    ax.errorbar(xs, normal_means_pct, yerr=[normal_lower_pct_errors, normal_upper_pct_errors], fmt=fmt[0], label=f"Honest Agents", markersize=12)
    ax.errorbar(xs, special_means_pct, yerr=[special_lower_pct_errors, special_upper_pct_errors], fmt=fmt[1], label=f"Malicious Agents", markersize=12)
    if legend:
        ax.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(30,5), sharey=True)

plot(ax1, "Adult - Random Agents", adult_random, ylabel=True)
plot(ax2, "KDD - Random Agents", kdd99_random)
plot(ax3, "Adult - Inverted Agents", adult_inverted)
plot(ax4, "KDD - Inverted Agents", kdd99_random, legend=True)

# plot(ax4, , title="KDD - Model ($m_k$)", series="model_scores")
# plot(ax5, kdd_client_data, title="KDD - Evaluation ($d_k$)", series="auditor_score")
# plot(ax6, kdd_client_data, title="KDD - Overall ($p_k$)", series="overall_score")
plt.tight_layout()
plt.savefig("exp4.pdf")
plt